# Introduction

Welcome to Crispr! See "Major Methods" below for an overview of a typical workflow. 

## Table of contents:
  * <a href=#Setup>Setup</a>
    * <a href=#Ojbect>Initialization of Object</a>
    * <a href=#Processing>Preprocessing & Clustering</a> 
  * <a href=#Plots>Initial Plotting</a>
  * <a href=#Analyses>Perturbation Analyses</a>
    * <a href=#Augur>Aggregate Cell-Type Perturbation Scoring (Augur)</a>
    * <a href=#Mixscape>Cell-Level Perturbation Classification & Response Scoring (Mixscape)</a>
    * <a href=#Distance>Distance Metrics</a>
    * <a href=#Composition>Compositional Analysis (scGen)</a>
    * <a href=#Dialogue>Multi-Cellular Programs (Dialogue)</a>

The `crispr.Crispr()` class is the main way of interacting with the package; however, if you want to manipulate a data object before subjecting it to the initial processing steps in the `Crispr.__init__` method for some reason, you can use `adata = cr.pp.create_object(file_path, ...)`, then pass `adata` to the class initialization method in lieu of a file/dictionary of file processing arguments.

<u> Argument Conventions: </u>

Certain arguments used throughout the `crispr` package (including outside the `crispr.crispr_class.Crispr()` class), hold to conventions intended to foster readability, maintain consistency, and promote clarity, both for end-users and future developers.

* Use the `copy` argument to avoid altering internal attributes (including `.adata`) in most `Crispr` class methods.

* Arguments starting in `col_` and `key_`
    - The "col_" prefix indicates that an argument refers to a column name (often in `.adata.obs` or `.adata.var`), while the "key_" prefix means you're meant to specify a type of entry in a column. For instance, assume the column "condition" contains the names of different experimental conditions (drug A, drug B, drug C). In a function where you want to compare, for instance, drug A vs. control, you would specify `key_treatment="drug A"` and k
    - These names may 
        * already exist (or will exist in the `.adata` attribute immediately upon creating the AnnData object from the data file) or 
        * may yet to be created, namely, after object initialization by running the object's methods. Thus, you may specify what you want certain columns to be named (e.g., the binary perturbed/non-perturbed column) or what entries within a column will be called (e.g., "Control" for rows within the `col_control` corresponding to cells that have control guide RNA IDs in `col_guide_rna`), for aesthetics, customizability to your design/interpretability, and/or to avoid duplicating pre-existing names.
    - These arguments will be entered as items (with the argument names as keys) in dictionaries stored in the object attributes `._columns` and `._keys`, respectively.
    - These arguments will often be passed by default (or will force them as specifications) to various object methods.
    - In certain methods, you can specify a new column to use just for that function. For instance, if you have a column containing CellTypist annotations and want to use those clusters instead of the "leiden" ones for the `run_dialogue()` method, you can specify in that method (`run_dialogue(col_cell_type="majority_voting")`) without changing the attribute (`self._columns`) that contains your original specification here.

<!-- break -->

* `col_perturbed` (binary) vs. `col_condition` (can have >= 3 categories)
    - In the `Crispr` class object, `col_perturbed` is meant to be a binary column that has `key_control` as the entry for control rows and `key_treatment` for all other experimental conditions. 
        * For instance, for a CRISPR design targeting more than one gene, `col_perturbed` would contain only `key_treatment` (i.e., all perturbed cells, regardless of targeted gene) while `col_condition` would contain entries specifying the particular gene(s) targeted (or `key_control`).
        * A drug design targeting more than one gene, `col_perturbed` would contain only `key_treatment` (i.e., all perturbed cells, regardless of targeted gene) while `col_condition` would contain entries specifying the particular gene(s) targeted (or `key_control`).
        * If the design only targets one gene/has one treatment conditions/etc., these columns would simply be equivalent.
    - In the `Crispr` class object, it is created during object initialization as a column (named after your specification of `col_perturbed`) in `.obs`. All rows in `.obs[col_condition]` that do not = `key_control` will be set as `key_treatment`.
    - In the `crispr` package more broadly, if a function calls for a `col_perturbed` argument, that indicates that it works with binary categories only. If it is fed a column with three or more categories, it will either subset the data to include only rows where that column = `key_treatment` or `key_control` (desirable behavior if you want to compare only a subset of the existing conditions, but undesirable if you want to look at, say, any drug vs. control, where the desired "drug" category consists of rows where the column = "drug A" and "drug B"), or it will throw an error.

## Initialization Method Arguments (Select)

Here are some of the important argument for the `Crispr()` class initialization method:

* `file_path`: Path or object containing data. Used in initialization to create the initial `self.adata` attribute (an AnnData or MuData object). Either
    - a path to a 10x directory (with matrix.mtx.gz, barcodes.tsv.gz, features.tsv.gz),
    - a path to an .h5ad or .mu file (Scanpy/AnnData/Muon-compatible), 
    - an AnnData or MuData object (e.g., already loaded with Scanpy or Muon, or by using `crispr.pp.create_object(file_path)`), or
    - a dictionary containing keyword arguments to pass to  `crispr.pp.combine_matrix_protospacer()` (in order to load information about perturbations from other file(s); see function documentation), or
    - to concatenate multiple datasets, a dictionary (keyed by your desired subject/sample names to be used in `col_sample_id`) consisting of whatever objects you would pass to `create_object()`'s `file` argument for the individual objects. You must also specify `col_sample` (a tuple as described in the documentation below). The other arguments passed to the `crispr.pp.create_object()` function (e.g., `col_gene_symbols`) can be specified as normal if they are common across samples; otherwise, specify them as lists in the same order as the `file` dictionary. 
* `col_sample_id`: Column in `.obs` with sample IDs. If this column does not yet exist in your data and needs to be created by concatenating datasets, you must provide
    - `file_path` as a dictionary keyed by desired `col_sample_id` values as well as 
    - signal that concatenation needs to happen by specifying `col_sample_id` as a tuple, with the second element containing a dictionary of keyword arguments to pass to `AnnData.concatenate()` or None (to use defaults).

See docstring in `crispr.crispr_object` and the README for more details.


## Crispr Object Properties

Below are descriptions of major `Crispr()` object attributes and methods.

### Attributes

* `.adata`: AnnData object. Columns or other objects created in the course of running certain methods may also be stored in its various attributes. Below are listed some of the major attributes of `.adata`. Note that for multi-modal data (self._assay is not None), some of these attributes may need to be accessed by `.adata[self._assay].<attribute>`, but for brevity, we'll refer to `.adata` here. Not all will/have to be present, except `.X`, `.obs`, and `.var`. You can access the gene expression modality directly by calling `self.rna`, whether the data are multi- or single-modal.
    -  `.X`: Sparse matrix of data originally passed to the function to create an AnnData object (e.g., from CellRanger output).
    -  `.layers`: Contains different versions of `adata.X`, such as scaled data (`adata.layers["scaled"]`) or that created by calculating a 
                  Mixscape perturbation signature (`adata.layers["X_pert"]`, by default).
    -  `.obs`: pandas.DataFrame of observations (rows=cells). You can store additional data about individual cells/samples/etc. here by assigning a new column. Can also be called directly using `self.obs`.
    -  `.obsm`: xxxxxxxxxxxxxxxxxxxxxxxxxx
    -  `.obsm`: xxxxxxxxxxxxxxxxxxxxxxxxxx
    -  `.var`: pandas.DataFrame of observations (rows=cells). You can store additional data about individual cells/samples/etc. here by assigning a new column. 
            Often contains the gene symbols and EnsemblIDs (either of which is often the index/`.var_names`),
            "feature_types" (e.g., "Gene Expression"), and, after preprocessing, may contain columns such as the number of cells expressing that feature ("n_cells"),
            whether that feature is a mitochonrial ("mt") and/or highly variable ("highly_variable") gene, mean and total counts, percent dropout, means, dispersions, 
            and normalized versions of these metrics. Can also be called directly using `self.var`.
    -  `.uns`: A dictionary that holds various results and other information. Can also be called directly using `self.uns`.
    -  `.obs_names`: Row indices of `.obs` (e.g., cell barcodes). Changing this attribute changes this index, and has other potential benefits/consequences.
    -  `.var_names`: Row indices of `.var` (i.e., gene names). Changing this attribute changes this index, and has other potential benefits/consequences.
    -  `.n_obs`: Number of observations (i.e., cells).
    -  `.n_vars`: Number of features (i.e., genes/proteins/etc.).
* `.results`: Stored output (other than figures). Note that some results may also be saved in `.obs` and `.uns` attributes of `.adata`.
* `.figures`: Stored figure objects.
* `.info`: Miscellaneous information about the object (e.g., its analysis/processing history).

### Methods

Here are the methods (applicable to scRNA-seq generally, not just perturbations) **in order** of a typical workflow _(replace ... with argument specifications)_:

* `self.preprocess(...)`: Perform filtering, normalization, scaling, quality control analysis, selection for highly variable genes, regressing out confounds, and intial exploration of the data (e.g., cell counts, mitochondrial counts, etc.).
* `self.cluster(...)`: Perform dimensionality reduction (e.g., PCA) and clustering (Louvain or Leiden), as well as related visualization.
* `self.plot(...)`: Create additional basic plots (e.g., dot, matrix, and violin gene expression plots).

The following perturbation-specific methods can be executed optionally and in any order:

* `self.run_augur(...)`: Score and plot how strongly different cell types responded to perturbation(s). This score is operationalized as the accuracy with which a machine learning model can use gene expression data to predict the perturbation condition to which cells of a given type belong. Augur provides scores aggregated across cells of a given type rather than for individual cells.
* `self.run_mixscape(...)`: Quantify and plot the extent to which individual cells responded to CRISPR perturbation(s), and identify which perturbation condition cells were not detectibly perturbed in terms of their gene expression.
* `self.compute_distance(...)`: Calculate and visualize various distance metrics that quantify the similarity in gene expression profiles across perturbation conditions.
* `self.run_composition_analysis(...)`: Analyze and visualize shifts in the cell type composition across perturbation conditions.
* `self.run_dialogue(...)`: Create plots showing multi-cellular programs.

# Setup

First, we need to initialize the object, and if the object isn't already preprocessed and/or clustered, perform those steps.

## Imports & Options

This first chunk contains code that a typical user would not need and is only included because we need to set initialization keywords and other variables (drawn from the `crispr/examples/config.py` file) that correspond to the dataset specified in the `file` variable. See `config.py` for the names of different files/datasets you can specify.

In [194]:
%load_ext autoreload
%autoreload 2

import crispr as cr
from crispr import Crispr
import pertpy as pt
import scanpy as sc
import pandas as pd
import numpy as np
from config import (
    files_data, assays_data, col_cell_type_data, col_gene_symbols_data,
    col_perturbed_data, col_condition_data, key_control_data,
    key_treatment_data, col_split_by_data, col_batch_data, col_sample_id_data,
    kws_process_guide_rna_data, col_guide_rna_data, col_num_umis_data)

pd.options.display.max_columns = 100

# Options
skip_preprocessing_clustering = False  # to skip these steps
print(dir(cr.ax))
file = "CRISPRi_scr"
# file = "CRISPRi_ess"
# file = "ECCITE"
# file = "augur_ex"
# file = "coda"
# file = "screen"
# file = "perturb-seq"

#  Set Arguments
kws_umap = dict(min_dist=0.3)  # passed to UMAP fx
assay, assay_protein = None, None  # will change below if needed
if file in assays_data and assays_data[file] is not None:
    assay, assay_protein = [assays_data[file], None] if isinstance(
        assays_data[file], str) else assays_data[file]
else:
    assay, assay_protein = None, None
col_split_by = col_split_by_data[file]
kwargs_init = dict(assay=assay, assay_protein=assay_protein,
                   col_gene_symbols=col_gene_symbols_data[file],
                   col_cell_type=col_cell_type_data[file],
                   col_sample_id=col_sample_id_data[file],
                   # col_batch=col_batch_data[file],
                   col_perturbed=col_perturbed_data[file],
                   col_guide_rna=col_guide_rna_data[file],
                   col_num_umis=col_num_umis_data[file],
                   kws_process_guide_rna={**kws_process_guide_rna_data[file]},
                   col_condition=col_condition_data[file],
                   key_control=key_control_data[file],
                   key_treatment=key_treatment_data[file])
if file == "default_fast":
    skip_preprocessing_clustering = True
if isinstance(file, str) and file in ["ECCITE", "Adamson"]:
    file_path = pt.data.papalexi_2021() if (
        file == "ECCITE") else pt.data.adamson_2016_upr_perturb_seq()
else:
    file_path = files_data[file]
print(file_path)
print(f"Initialization arguments (from config):\n\n{kwargs_init}")
print(f"UMAP Keywords: {kws_umap}\n\n")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'analyze_causal_network', 'analyze_composition', 'analyze_receptor_ligand', 'calculate_dea_deseq2', 'cluster', 'clustering', 'communication', 'composition', 'compute_distance', 'find_marker_genes', 'perform_augur', 'perform_celltypist', 'perform_dea', 'perform_differential_prioritization', 'perform_gsea', 'perform_mixscape', 'perform_pathway_interference', 'perturbations']
{'directory': '/home/elizabeth/elizabeth/crispr/examples/data/crispr-screening/HH06'}
Initialization arguments (from config):

{'assay': None, 'assay_protein': None, 'col_gene_symbols': 'gene_symbols', 'col_cell_type': 'predicted_labels', 'col_sample_id': None, 'col_perturbed': 'perturbation', 'col_guide_rna': 'feature_call', 'col_num_umis': 'num_umis', 'kws_process_guide_rna': {'feature_split': '|', 'guide_split'

[autoreload of crispr.processing.importing failed: Traceback (most recent call last):
  File "/home/elizabeth/elizabeth/miniconda3/envs/py-bio/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/elizabeth/elizabeth/miniconda3/envs/py-bio/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/elizabeth/elizabeth/miniconda3/envs/py-bio/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/elizabeth/elizabeth/crispr/cr

## Object

This code instantiates the CRISPR object, which is the main way of interacting with this package as an end-user.

This is more code than you would need in real life; it just ensures that certain public datasets are loaded from the source for various reasons.

In [195]:
self = Crispr(file_path, **kwargs_init)
self.rna.obs  # RNA assay/gene expression modality dataframe
adata = self.adata.copy()



<<< INITIALIZING OMICS CLASS OBJECT >>>

Unused keyword arguments: {'kws_process_guide_rna': {'col_guide_rna': 'feature_call', 'col_num_umis': 'num_umis', 'key_control': 'NT', 'col_guide_rna_new': 'target_gene_name', 'feature_split': '|', 'guide_split': '-', 'key_control_patterns': ['CTRL'], 'remove_multi_transfected': True, 'min_n_target_control_drop': None, 'max_pct_control_drop': 75, 'min_pct_dominant': 80, 'min_pct_avg_n': 40}}.

col_gene_symbols="gene_symbols"
col_cell_type="leiden"
col_sample_id=None
col_batch=None
col_subject=None
col_condition="target_gene_name"
col_num_umis="num_umis"
key_control="NT"
key_treatment="KD"


<<< LOADING PROTOSPACER METADATA >>>


KeyboardInterrupt: 

## Preprocessing

In [ ]:
if skip_preprocessing_clustering is False:
    process_kws = dict(
        kws_hvg=dict(min_mean=0.0125, max_mean=10, min_disp=0.5,
                     flavor="cell_ranger"),  # for highly-variable gene fx
        target_sum=1e4,  # total-count normalization (1e4=10,000/cell)
        cell_filter_pmt=[0, 30],  # range of mitchondrial gene % allowed
        cell_filter_ngene=[0, 7000 if isinstance(file, str) and (
          "CRISPRi_scr" in file) else None],  # cells: min/max genes expressed
        cell_filter_ncounts=[3, 60000 if isinstance(file, str) and (
          "CRISPRi_scr" in file) else 40000],  # cells: min/max read range
        gene_filter_ncell=[3, None],  # genes: min/max cell count range
        regress_out=None,  # don't regress out confounds like pct_mt
        kws_scale="z")  # z-score w.r.t control means/SDs
        # kws_scale=True)  # or just overall center/standardize GEX
        # kws_scale=10)  # (use integer to also place a max_value)
    print(process_kws)
    self.preprocess(**process_kws, kws_umap=kws_umap)  # preprocessing
    self.rna.obs

## Clustering

In this section, we'll perform steps related to clustering/annotation of cell types.

### Leiden Clustering

Let's perform dimensionality reduction (PCA) and cluster the data using the "leiden" method. 

In [ ]:
if skip_preprocessing_clustering is False:
    self.cluster(method_cluster="leiden", resolution=0.5,
                 kws_pca=dict(n_comps=None, use_highly_variable=True),
                 kws_umap=kws_umap, kws_neighbors=None, kws_cluster=None)

### Find Cluster Markers (DEGs)

To sort by absolute values on "scores" for cluster 6, showing the top 3 gene markers:

`clus = "6"`

`marks.loc[clus].loc[marks.loc[clus].scores.abs().sort_values(ascending=False).iloc[:3].index]`

To show the top 3 gene markers for all clusters:

`marks.groupby(marks.index.names[0]).apply(lambda x: x.scores.abs().sort_values(ascending=False).iloc[:3].index])`

In [ ]:
degs = self.find_markers(n_genes=10, method="wilcoxon", key_reference="rest")
degs[0]  # degs[1] for figures

### CellTypist Annotations

Now let's detect cell types using `CellTypist`.

You can provide a CellTypist model name as a string, or **to train your own custom model**, provide an `AnnData` object with the training data and, if cell type column names differ across training and main data (in `self`) or if you want to pass additional arguments to `celltypist.train()`, then provide a dictionary of arguments to `kws_train`. (Cell type name can be under `labels` or `col_cell_type` in this dictonary.)

**We will use an aribtrary model just for illustration, but you should pick one that suits your data.**

You can see what models are available with this code:
```
import celltypist
celltypist.models.models_description().set_index("model").description.apply(lambda x: print(x))
celltypist.models.models_description()
```

Or it will show you valid models automatically after the error message if you feed it an unknown model:

```
self.annotate_clusters(model="kate_moss.pkl")
```

In [ ]:
_ = self.annotate_clusters("COVID19_HumanChallenge_Blood.pkl")
# THIS MODEL IS JUST FOR ILLUSTRATION
# YOU SHOULD CHOOSE ONE THAT MAKES SENSE FOR YOUR DATA
self.rna.obs

# Inspection

First, you can easily print the `.adata` representation, the gene expression modality `.obs` preview, and columns and keys stored in the object's attributes using the `print()` method.

In [ ]:
self.print()

## Set Up Arguments for Later


This code looks more complicated than it actually would actually be for an end user because it was made to be generalizable across several datasets with particular column names, sizes that make it necessary to subset them in order to run the vignettes in a reasonable period of time, etc.

Basically, you won't need this code as an end user; this is just to choose random subsets of genes and perturbations, etc. that are available in a given example dataset.

In real use cases, you will know what genes and conditions are of interest, and you can manually specify them by simply stating them in the appropriate arguments (such as `target_gene_idents`) or (in many cases) by not specifying the argument (resulting in the code using all available genes, etc.).

In [ ]:
genes = self.rna.var.reset_index()[self._columns["col_gene_symbols"]]
if file == "CRISPRi_scr":
    genes_subset = [
        "CDKN2A", "CDKN1A", "PLAUR", "TP53", "IL6ST", "IL1A", "CXCL8", "CCL2",
        "CEBPB", "NFKB1", "IGFBP7", "BCL2", "ERCC1", "ERCC4", "PTGER4",
        "TGFB1", "ATG16L1", "NOD2", "LACC1"]
    genes_subset = list(set(genes_subset).intersection(genes))
else:
    genes_subset = list(pd.Series(genes).sample(10))
target_gene_idents = list(self.obs[self._columns[
    "col_target_genes"]].sample(10))  # 10 random guide gene targets

## Explore Data Descriptives

You can plot and explore descriptives using the `describe()` method.

In [ ]:
_ = self.describe()  # simple
# _ = self.describe(group_by=self._columns["col_target_genes"], plot=True)

# Plots

Create plots applicable to scRNA-seq data broadly (e.g., UMAP, dotplots) without having to do any perturbation-specific analyses.

## Basic Usage

You can create simple plots easily without having to remember a bunch of arguments to specify! 

The most useful is the `genes` argument, which allows you to subset the number of features plotted (useful for spead and layout/interpretability of plots).

In [ ]:
# figs = self.plot(genes=["gene A", "gene B"...])  # to specify specific genes
figs = self.plot(genes=24)  # to specify random subset of this # of genes

## Advanced Usage

Use the `layers` argument to plot more layers (in this instance, all of them, including the scaled data) for certain plot types.
 
Use the `cell_types_circle` argument to create a UMAP with certain cell types circled in red.

Use the `genes_highlight` argument to highlight in gold the names of specified genes on the gene expression dot plot.

Use the `kws_clustering` argument to specify a dictionary keywords to pass to certain UMAP-based plots. For instance, specify `kws_clustering=dict(col_cell_type="leiden")` to use leiden clusters instead of whatever is stored in `self._columns["col_cell_type"]`.

Use the `kws_gex_violin` argument to pass additional arguments to the violin plots of gene expression.

In [ ]:
cct = "predicted_labels" if "predicted_labels" in self.rna.obs else None
clus = list(self.rna.obs[cct].sample(2))  # clusters to circle
figs = self.plot(genes=36,
                 col_cell_type=cct,
                 cell_types_circle=clus,  # list cell types to circle on UMAP
                 genes_highlight=list(np.array(genes_subset)[1:3]),
                 kws_gex_violin=dict(scale="area", height=10),
                 kws_umap=dict(col_cell_type=cct))

## Specific Plots

Use the package visualization sub-module function to make specific, customizable plots. 

Use arguments creatively to customize. For example, set the `col_cell_type` argument to the name of the perturbation condition column in the `plot_gex()` function to plot gene expression across perturbation conditions instead of cell types.

Use `kws_<kind>` (replacing <kind> with the plot type, e.g., `kws_dot`) arguments to specify a dictionary of keyword arguments to pass to a Scanpy plotting functions (e.g., https://scanpy.readthedocs.io/en/stable/generated/scanpy.pl.dotplot.html) for further customization.

In [ ]:
# Scanpy Figure Defaults
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=200, frameon=True,
                     fontsize=8, figsize=(20, 40), color_map=None,
                     format="pdf", facecolor=None, transparent=False)

# Simple GEX Heatmap by Cell Type
fig = self.plot(kind="hm", genes=genes_subset, figsize=(60, 60))

# Argument to Subset Data to Target Genes of Interest
target_gene_idents = ["NOD2", "LACC1", "ATG16L1", "SP100", "NT"]
subset = list(self.rna.obs[self._columns["col_condition"]].isin(
    target_gene_idents))

# Dot Plot
fig = self.plot(["FTL", "FTH1", "STAT1", "CCL5"],
                subset=subset, kind="dot",  # specify dot plot
                col_cell_type=self._columns["col_condition"])

# Dot Plot with Genes Grouped & Labeled
marker_genes_dict = {"Monocyte": ["FCGR3A", "FCN1", "S100A4", "S100A6"],
                     "B": ["CD79A", "CD19"]
                     }  # labeled ~ cell type here, but labels can be anything
fig = self.plot(subset=subset, kind="dot",  # specify dot plot
                col_cell_type=self._columns["col_condition"],
                marker_genes_dict=marker_genes_dict)

# Heatmap
fig = self.plot(subset=subset, kind="hm",  # specify heatmap plot
                col_cell_type=self._columns["col_condition"],
                genes=genes_subset, figsize=(60, 60), layer="scaled")

# Matrix Plot
fig = self.plot(subset=subset, kind="matrix",  # specify heatmap plot
                col_cell_type=self._columns["col_condition"],
                genes=genes_subset, layer="scaled", figsize=(60, 60))

# Analyses

The following examples concern CRISPR or other perturbation design-specific analyses.

## Guide RNA Counts/Percentage

In [ ]:
# Choose Subset of Target Genes (optional)
tgis = list(pd.Series(target_gene_idents).sample(3)) if len(
    target_gene_idents) > 3 else target_gene_idents  # smaller subset = faster
cct = "majority_voting" if "majority_voting" in self.rna.obs else None

# Guide RNA Counts
_ = self.get_guide_rna_counts(target_gene_idents=tgis)

# ...By Cell Type
_ = self.get_guide_rna_counts(target_gene_idents=tgis, group_by=cct)

# # ...By Target Gene Ultimately Assigned
# _ = self.get_guide_rna_counts(target_gene_idents=tgis,
#                               group_by=self._columns["col_target_genes"])

# ...By Cell Type & Target Gene Ultimately Assigned
_ = self.get_guide_rna_counts(target_gene_idents=tgis, group_by=[
    cct, self._columns["col_target_genes"]], margin_titles=True)

## Distance

Distance Metrics

See `self.figures["distances"]` and  `self.results["distances"]` for results.

In [ ]:
for x in ["mmd", "edistance"]:
    _ = self.compute_distance(distance_type=x, method="X_pca", kws_plot=dict(
        figsize=(15, 15), robust=True))

## Augur: Perturbation Responses by Cell Type

**Which cell types are most affected by perturbations?** Quantify perturbation responses by cell type with Augur, which uses supervised machine learning classification of experimental condition labels (e.g., treated versus untreated). The more separable the condition among cells of a given type, the higher the perturbation effect score.

<u> __Features__ </u>  

- Quantify and visualize degree of perturbation response by cell type
- Identify the most important features (genes).

<u> __Input__ </u>  

There are no required arguments. 
* If you want to override defaults drawn from `._columns` and/or `._keys`, specify the appropriate argument (e.g., `col_cell_type`). 
* You can also specify a different `classifer` (default "random_forest_classifer") used in the machine learning classification procedure used to calculate the AUCs/accuracy. 
* You may pass keyword arguments to the Augur predict method by specifying a dictionary in `kws_augur_predict`.
* Specify `select_variance_features` as True to run the original Augur implementation, which removes genes that don't vary much across cell type. If False, use features selected by `scanpy.pp.highly_variable_genes()`, which is faster and sensitively recovers effects; however, the feature selection may yield inflated Augur scores because this reduced feature set is used in training, resulting in it taking advantage of the pre-existing power of this feature selection to separate cell types.
* Specify `n_folds` and/or `subsample_size` to choose the number and sample size of folds in cross-validation.
* Set an integer for `seed` to allow reproducibility across runs.

<u> __Output__ </u>  

Tuple, where the first element is the AnnData object created by the function, the second, the results dictionary, and the third, a dictionary of figures visualizing results. If copy is False (default), these outputs can also be found in `.results["augur"]` and `.figures["augur"]`.

<u> __Notes__ </u>  

- Sub-sample sizes equal across conditions; does not account for perturbation-induced compositional shifts (cell type abundance)
- Scores are for cell types (aggregated across cells, not individual cells)
- Two modes
    - If select_variance_feature=True, 
    - If False, you also have to be sure that "highly_variable_features" is a variable in your data. This can be complicated if you have a separate layer for perturbation data.

In [ ]:
cct = "majority_voting" if "majority_voting" in self.rna.obs else \
    self._columns["col_cell_type"]
_ = self.run_augur(
    col_cell_type=cct,
    # ^ will be label in self._columns by default, but can override here
    col_perturbed=self._columns["col_perturbed"],
    # ^ will be this by default if unspecified, but can override here
    key_treatment=self._keys["key_treatment"],
    # ^ will be this by default if unspecified, but can o verride here
    select_variance_features=True,  # filter by highly variable genes
    classifier="random_forest_classifier", n_folds=3, augur_mode="default",
    kws_umap=kws_umap, subsample_size=5, kws_augur_predict=dict(span=0.7),
    n_threads=None)

## Mixscape: Cell-Level Perturbation Classification & Scoring

**Is a perturbed cell detectibly perturbed, and to what extent?** Mixscape first calculates the "**perturbation signature**" by determining which control condition cells most closely resemble each perturbed cell in terms of mRNA expression and then subtracts the control expression from that of the perturbed cells' (i.e., centers perturbed cells' gene expression on their control neighbors).

Then, it **identifies** and removes perturbed **cells with no detectible perturbation** (i.e., assigns them to predicted classes of perturbed versus not perturbed). You can then create visuals based on whether the cell is detectibly perturbed, "non-perturbed" (not detectibly perturbed), or control (no treatment). Optionally, you can visualize protein expression by this predicted class in certain multi-modal data.

**Are there perturbation-specific clusters?** Mixscape uses linear discriminant analysis (LDA) to cluster cells that resemble each other in terms of gene expression and perturbation condition. _(LDA reduces dimensionality and attempts to maximize the separability of classes. Unperturbed cells are removed from analysis.)_ 

<u> __Features__ </u>  

- Plot targeting efficiency.
- Remove confounds (e.g., cell cycle, batch effects)
- Classify cells as affected or unaffected (i.e., "escapees") by the perturbation
- Quantify and visualize degree of perturbation response

<u> __Input__ </u> 

See documentation, but the key arguments are listed here.

* **col_cell_type**: If you want to run using a different cell classification column, (e.g., CellTypist annotations that weren't used for the original `self._columns["col_cell_type"]``), you can specify a different column by passing `col_cell_type=<column name>` if you'd like.
* **target_gene_idents**: A list of gene symbols to focus on in plots/analyses. Specify as True to include all.
* **target_gene_idents**: The default layer of data used is "log1p." Remember that Mixscape centers cells on their control neighbors when considering whether to use centered and/or scaled data.

<u> __Output__ </u>  

Assuming your `Crispr` object is named "self":
- Targeting Efficiency: `self.figures["mixscape"]["targeting_efficiency"]`
- Differential Expression Ordered by Posterior Probabilities: `self.figures["mixscape"]["DEX_ordered_by_ppp_heat"]`
- Posterior Probabilities Violin Plot: `self.figures["mixscape"]["ppp_violin"]`
- Perturbation Scores: `self.figures["mixscape"]["perturbation_score"]`
- Perturbation Clusters (from LDA): `self.figures["mixscape"]["perturbation_clusters"]`

The above instructions are for accessing output via the object attributes. Assuming output is assigned to a variable `figs` (i.e., `figs = ` would replace the `_ = ` in the code below), replace `self.figures["mixscape"]` in the above code with `figs`.

<u> __Notes__ </u>  

- If `._columns["col_sample_id"]` is not None, perturbation scores will by default be calculated and/or plotted taking that into consideration (e.g., biological replicates) unless `col_split_by=False`. That argument can also be set to a different column name explicitly, in which case that specification will be used as the `col_split_by` argument in Pertpy Mixscape functions in place of sample ID.

#### Run Mixscape

In [ ]:
cct = "predicted_labels" if "predicted_labels" in self.rna.obs else None
tgis = list(self.rna.obs[self._columns["col_target_genes"]].sample(3))
_ = self.run_mixscape(col_cell_type=cct, target_gene_idents=tgis)
# _ = self.run_mixscape(target_gene_idents=True)  # plot all target genes

#### Create Different Mixscape Plots

If you want to re-create mixscape **plots with <u> different target genes and/or proteins of interest**</u> later, you can use `self.plot_mixscape(<ONE OR MORE TARGET GENES>)`. If you want a different color for the perturbation score curves, specify `color=` in that method.

In [ ]:
tgis = pd.Series(self.rna.uns["mixscape"].keys()).sample(1)
_ = self.plot_mixscape(tgis, color="red")

## Multi-Cellular Programs (Dialogue)

In [ ]:
cct = "majority_voting" if "majority_voting" in self.rna.obs else \
    self._columns["col_cell_type"]
fig_mcp = self.run_dialogue(n_programs=4, cmap="coolwarm")